#### Install langchain chroma for storage vector

In [ ]:
!pip install -qU langchain-chroma

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install selenium

In [ ]:
!pip install unstructured

#### Create a RAG System To Test

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import SeleniumURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_ollama import ChatOllama

In [14]:
## Setting up the Ollama LLM
llm = ChatOllama(
    model="deepseek-r1:8b",
    base_url="http://localhost:11434",
    temperature=0.5,
    num_predict=500
)

In [ ]:
## Loading data from web page
loader = SeleniumURLLoader(urls=["https://www.descope.com/learn/post/mcp"])
data = loader.load()
## Splitting the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=100, 
                                               separators=["\n\n", "\n", ". ", " ", ""])
splits = text_splitter.split_documents(data)
## Creating embeddings and vector store
embedding = OllamaEmbeddings(model="llama3.2:latest")
vector_db = Chroma.from_documents(splits, embedding)
## Setting up retrieval augmented generation
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
## Creating prompt template
template = """
    Answer the question based only on following context:
    {context}
    Give a summary not the full details
    Question: {question}
    Instructions:
        - Answer based ONLY on the information provided in the context above
        - If the context doesn't contain enough information to answer the question, say so
        - Be accurate and precise in your response
        - Give a summary, not full details
"""
prompt = ChatPromptTemplate.from_template(template)
## Function to format documents
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])
## Function to retrieve and format the prompt
def retrieve_and_format(question: str) -> List[Document]:
    docs = retriever.invoke(question)
    # print(f"Number of documents retrieved: {len(docs)}")
    # print("--- Retrieved Document Content ---")
    # for doc in docs:
    # # Print the content and metadata source
    #     print(f"Source: {doc.metadata.get('source', 'N/A')}")
    #     print(doc.page_content[:300] + "...\n")
    return format_docs(docs)
## Final runnable chain
rag_chain = {"context": retrieve_and_format, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [20]:
response = rag_chain.invoke("What is MCP")
print(response)

Based on the provided context, MCP (Model Context Protocol) is a protocol designed to connect AI applications like Claude Desktop to external tools and resources.

Key points from the summary:

1.  **Purpose:** To allow AI apps to utilize capabilities (tools, resources, prompts) offered by external servers.
2.  **Mechanism:** It involves a handshake/connection, capability discovery (asking the server what it offers), and registration of those capabilities.
3.  **Benefit:** It builds upon function calling (the standard way LLMs call


In [23]:
response = rag_chain.invoke("What is relationship between function calling and Model context protocol")
print(response)

Based on the provided context:

Function calling is the primary method used to call APIs from LLMs. MCP builds upon this concept by standardizing the way tools (functions) are specified, discovered, and executed across different AI systems and models. Essentially, MCP provides a universal protocol for implementing function-like capabilities in AI apps, making it compatible with various model vendors unlike some original function calling implementations which might have been more limited in scope.


In [25]:
response = rag_chain.invoke("What is the Architecture of MCP")
print(response)

Based on the provided context, here is a summary of the MCP architecture:

The MCP architecture involves an **AI application** (like Claude Desktop) acting as the **Client**, and the **external tools/resources/APIs** acting as the **Servers**.

The core process is a **Protocol Handshake**:

1.  **Initial Connection:** The client connects to the configured MCP servers.
2.  **Capability Discovery:** The client asks the server what capabilities/tools/resources it offers.
3.  **Registration:** The client registers the discovered capabilities, making them available for the AI to use.

This architecture connects AI applications to external context and builds upon standard function calling methods to simplify and standardize development, aiming to solve issues like excessive maintenance and fragmented implementation.


#### Testing RAG Application